In [1]:
import sys
import os

# Add the path to `database_scripts` folder
sys.path.append(os.path.abspath("../database_scripts"))
import db_utils as du
import create_tables as ct


conn = ct.create_or_open_database()
df_fahrrad_Zähler = du.fetch_data_df('Standorten_Zählstelle',conn)
df_mess_Fahrrad= du.fetch_data_df('Messdaten_Fahrrad',conn)
df_bezirke= du.fetch_data_df('Bezirke',conn)
df_date = du.fetch_data_df('Date_dim',conn)
df_time = du.fetch_data_df('Time_dim',conn)
df_wetter = du.fetch_data_df('Wetter',conn)

Establish SQLite connection.


In [6]:
df_mess_Fahrrad

,Zählstelle,DateID,TimeID,Wert
0,02-MI-JAN-N,1,0,12.0
1,02-MI-JAN-N,1,1,17.0
2,02-MI-JAN-N,1,2,25.0
3,02-MI-JAN-N,1,3,18.0
4,02-MI-JAN-N,1,4,15.0
...,...,...,...,...
4995691,27-RE-MAR,2191,19,18.0
4995692,27-RE-MAR,2191,20,11.0
4995693,27-RE-MAR,2191,21,10.0
4995694,27-RE-MAR,2191,22,5.0


In [5]:
df_bezirke

,Bezirk,Breitengrad,Längengrad,Geometry
0,Charlottenburg-Wilmersdorf,52.501049,13.270236,"POLYGON ((13.1881678 52.4718369, 13.1882878 52..."
1,Friedrichshain-Kreuzberg,52.503831,13.428232,"POLYGON ((13.3685807 52.4939555, 13.3684148 52..."
2,Lichtenberg,52.534895,13.509767,"POLYGON ((13.5169886 52.5354135, 13.517064 52...."
3,Marzahn-Hellersdorf,52.521115,13.578833,"POLYGON ((13.5168837 52.5355476, 13.5170363 52..."
4,Mitte,52.532878,13.365911,"POLYGON ((13.4041908 52.5404013, 13.4047046 52..."
5,Neukölln,52.441087,13.454967,"POLYGON ((13.3999243 52.4185371, 13.4001226 52..."
6,Pankow,52.597193,13.440249,"MULTIPOLYGON (((13.3883808 52.5676794, 13.3884..."
7,Reinickendorf,52.595678,13.291329,"POLYGON ((13.3491334 52.5892003, 13.3497913 52..."
8,Spandau,52.525795,13.178813,"POLYGON ((13.1095585 52.4510789, 13.1097339 52..."
9,Steglitz-Zehlendorf,52.434849,13.239081,"POLYGON ((13.088345 52.4196325, 13.0958391 52...."


In [4]:
df_time

,TimeID,time_of_the_day
0,0,nachts
1,1,nachts
2,2,nachts
3,3,nachts
4,4,morgens
5,5,morgens
6,6,morgens
7,7,morgens
8,8,morgens
9,9,morgens


In [3]:
df_wetter

,DateID,TimeID,temperature_2m (°C),relative_humidity_2m (%),rain (mm),snowfall (cm),cloud_cover (%)
0,1,0,11.2,71,0.0,0.0,57
1,1,1,11.3,69,0.0,0.0,73
2,1,2,11.7,64,0.0,0.0,90
3,1,3,11.9,62,0.0,0.0,92
4,1,4,11.7,63,0.0,0.0,92
...,...,...,...,...,...,...,...
631003,2191,19,6.6,79,0.1,0.0,100
631004,2191,20,6.3,79,0.0,0.0,100
631005,2191,21,6.6,80,0.0,0.0,100
631006,2191,22,6.3,83,0.0,0.0,96


In [7]:
import pandas as pd
df_merged = pd.merge(df_fahrrad_Zähler, df_bezirke, on="Bezirk", how="inner")

df_merged = pd.merge(df_merged, df_mess_Fahrrad, on="Zählstelle", how="inner")

# Merge with Date_dim on Installationsdatum and Date
df_merged = pd.merge(df_merged, df_date, on="DateID", how="inner")

# Merge with time_dim on Installationsdatum time
df_merged = pd.merge(df_merged, df_time, on='TimeID', how="inner")

#Merge with Wetter
# wenn mit Durschnitt on join would be TimeID auch aber hier wir können auch nach Bezirk joinen
df_merged = pd.merge(df_merged, df_wetter, on='Bezirk', how="inner") 
# Display the merged da
df_merged.head()

KeyError: 'Bezirk'